<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

## Packages

In [1]:
!pip install --upgrade git+https://github.com/xiaoyufan/nbme.git

  Cloning https://github.com/xiaoyufan/nbme.git to /tmp/pip-req-build-hp_jjddi
  Running command git clone -q https://github.com/xiaoyufan/nbme.git /tmp/pip-req-build-hp_jjddi
  Created wheel for nbme: filename=nbme-1.0-py3-none-any.whl size=1876 sha256=6876ebcd812db9041bfcd15d1cb55860f14e7225af42fa077887da6bf04a0fae
  Stored in directory: /tmp/pip-ephem-wheel-cache-e1cqjiop/wheels/e2/dc/dd/b061e30220e414c0a096e4fbaddd405c1b526d1453d9a447e3
Successfully built nbme


In [2]:
from nbme_utils import scoring